In [44]:
import requests
import pyspark

In [4]:
api_key = "tBJsUHneEMoqQFLhpLYtHUHtZ"
BBOX = "[[[-25.9973998712, 25.3439083708], [44.6049090453, 71.2982931893]]]"

In [32]:
min_long = float(BBOX.split(", ")[0].strip("[[["))
max_long = float(BBOX.split(", ")[1].strip("]"))
min_lat = float(BBOX.split(", ")[2].strip("["))
max_lat = float(BBOX.split(", ")[3].strip("]]]"))

In [33]:
max_lat, min_lat, max_long, min_long

(71.2982931893, 44.6049090453, 25.3439083708, -25.9973998712)

In [62]:
url = (
    "https://api.marinesia.com/api/v1/port/nearby?"
    + f"lat_min={min_lat}&lat_max={max_lat}&long_min={min_long}&long_max={max_long}"
    + f"&key={api_key}"
)

In [63]:
ans = requests.get(url).json()

In [64]:
ans

{'error': True, 'message': 'Your account has been restricted'}

In [65]:
requests.get(
    "https://api.marinesia.com/api/v1/vessel/470000000/profile?key=tBJsUHneEMoqQFLhpLYtHUHtZ"
).json()

{'error': True, 'message': 'Your account has been restricted'}